In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import Preprocessing
import DataProcessing
import model
import numpy as np
import utils
import time
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
# load data 
data = Preprocessing.load_data('data/rt_iot2022', target_col='Attack_type')

c:\Users\QUIN\anaconda3\envs\PPNKCH\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
Preprocessing.check_types(data,"object")

proto
service
Attack_type


In [3]:
attack_labels = data['Attack_type'].unique()

In [4]:
data = Preprocessing.delete_columns(data, ["Unnamed: 0",'service'])
data = data.loc[:, data.columns.notna()]
data = Preprocessing.one_hot_encode(data, ['proto'])
data,_ = Preprocessing.label_encode(data, ['Attack_type'])
print(data.head(5))

   id.orig_p  id.resp_p  flow_duration  fwd_pkts_tot  bwd_pkts_tot  \
0      38667       1883      32.011598             9             5   
1      51143       1883      31.883584             9             5   
2      44761       1883      32.124053             9             5   
3      60893       1883      31.961063             9             5   
4      51087       1883      31.902362             9             5   

   fwd_data_pkts_tot  bwd_data_pkts_tot  fwd_pkts_per_sec  bwd_pkts_per_sec  \
0                  3                  3          0.281148          0.156193   
1                  3                  3          0.282277          0.156821   
2                  3                  3          0.280164          0.155647   
3                  3                  3          0.281593          0.156440   
4                  3                  3          0.282111          0.156728   

   flow_pkts_per_sec  ...      idle.max      idle.tot      idle.avg  idle.std  \
0           0.437341  .

In [5]:
# Tìm các cột không phải là số (numeric)
non_numeric_cols = data.select_dtypes(exclude=["number"]).columns.tolist()
print("Các cột không phải kiểu số:")
print(non_numeric_cols)


Các cột không phải kiểu số:
['proto_tcp', 'proto_udp']


In [6]:
#Tách cột target ra khỏi dữ liệu
X, y = data.drop('Attack_type', axis=1), data['Attack_type']
X,_ = Preprocessing.scale_data(X)
# X_balanced, y_balanced = Preprocessing.apply_smote(X, y)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [7]:
#chưa áp dụng smotesmote
X_train_nsm, X_test_nsm, y_train_nsm, y_test_nsm = train_test_split(X,y, test_size=0.2, random_state=42)

#áp dụng smote
X_train_sm, X_test_sm, y_train_sm, y_test_sm = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_sm, y_train_sm = Preprocessing.apply_smote(X_train_sm, y_train_sm)

Train với dữ liệu chưa giảm chiều

LinearSVC

In [ ]:
model_LinearSVC = model.model_LinearSVC(X_train_sm, y_train_sm)
y_pred = model_LinearSVC.predict(X_test_sm)
utils.report(y_test_sm, y_pred)
utils.plot_confusion_matrix(y_test_sm, y_pred, attack_labels, "Ma trận nhầm lẫn của dữ liệu chưa giảm chiều - Mô hình LinearSVC" )

In [ ]:
accuracy = accuracy_score(y_test_sm, y_pred)
print("Accuracy:", accuracy)

report = classification_report(y_test_sm, y_pred, output_dict=True)
print("Precision (macro):", report["macro avg"]["precision"])
print("Recall (macro):", report["macro avg"]["recall"])
print("F1-score (macro):", report["macro avg"]["f1-score"])
print("Support (macro):", report["macro avg"]["support"])

xgboost

In [ ]:
# import numpy as np

# label_mapping = {
#     'Attack_type': {
#         np.int64(0): 'ARP_poisioning',
#         np.int64(1): 'DDOS_Slowloris',
#         np.int64(2): 'DOS_SYN_Hping',
#         np.int64(3): 'MQTT_Publish',
#         np.int64(4): 'Metasploit_Brute_Force_SSH',
#         np.int64(5): 'NMAP_FIN_SCAN',
#         np.int64(6): 'NMAP_OS_DETECTION',
#         np.int64(7): 'NMAP_TCP_scan',
#         np.int64(8): 'NMAP_UDP_SCAN',
#         np.int64(9): 'NMAP_XMAS_TREE_SCAN',
#         np.int64(10): 'Thing_Speak',
#         np.int64(11): 'Wipro_bulb'
#     }
# }

# for key, value in label_mapping['Attack_type'].items():
#     print(f"{key} -> {value}")


In [ ]:
attack_labels = [
    'ARP_poisioning',
    'DDOS_Slowloris',
    'DOS_SYN_Hping',
    'MQTT_Publish',
    'Metasploit_Brute_Force_SSH',
    'NMAP_FIN_SCAN',
    'NMAP_OS_DETECTION',
    'NMAP_TCP_scan',
    'NMAP_UDP_SCAN',
    'NMAP_XMAS_TREE_SCAN',
    'Thing_Speak',
    'Wipro_bulb'
]

In [ ]:
print(y_test_sm.value_counts())

In [ ]:
model_xgboost = model.model_xgboost(X_train_sm, y_train_sm,12)
y_pred = model_xgboost.predict(X_test_sm)
utils.report(y_test_sm, y_pred)
 
utils.plot_confusion_matrix(y_test_sm, y_pred, attack_labels, "Ma trận nhầm lẫn của dữ liệu chưa giảm chiều - Mô hình XGBoost" )

In [ ]:
accuracy = accuracy_score(y_test_sm, y_pred)
print("Accuracy:", accuracy)

report = classification_report(y_test_sm, y_pred, output_dict=True)
print("Precision (macro):", report["macro avg"]["precision"])
print("Recall (macro):", report["macro avg"]["recall"])
print("F1-score (macro):", report["macro avg"]["f1-score"])
print("Support (macro):", report["macro avg"]["support"])

logistic_regression

In [ ]:
model_logistic_regression = model.model_logistic_regression(X_train_sm, y_train_sm)
y_pred = model_logistic_regression.predict(X_test_sm)
utils.report(y_test_sm, y_pred)
utils.plot_confusion_matrix(y_test_sm, y_pred, attack_labels, "Ma trận nhầm lẫn của dữ liệu chưa giảm chiều - Mô hình Logistic regression" )

In [ ]:
accuracy = accuracy_score(y_test_sm,y_pred)
print("Accuracy:", accuracy)

report = classification_report(y_test_sm,y_pred, output_dict=True)
print("Precision (macro):", report["macro avg"]["precision"])
print("Recall (macro):", report["macro avg"]["recall"])
print("F1-score (macro):", report["macro avg"]["f1-score"])
print("Support (macro):", report["macro avg"]["support"])

KNN

In [ ]:
model_knn = model.model_knn(X_train_sm, y_train_sm)
y_pred = model_knn.predict(X_test_sm)
utils.report(y_test_sm, y_pred)
utils.plot_confusion_matrix(y_test_sm, y_pred, attack_labels, "Ma trận nhầm lẫn của dữ liệu chưa giảm chiều - Mô hình KNN" )

In [ ]:
accuracy = accuracy_score(y_test_sm, y_pred)
print("Accuracy:", accuracy)

report = classification_report(y_test_sm, y_pred, output_dict=True)
print("Precision (macro):", report["macro avg"]["precision"])
print("Recall (macro):", report["macro avg"]["recall"])
print("F1-score (macro):", report["macro avg"]["f1-score"])
print("Support (macro):", report["macro avg"]["support"])

random_forest

In [ ]:
model_random_forest = model.model_random_forest(X_train_sm, y_train_sm)
y_pred = model_random_forest.predict(X_test_sm)
utils.report(y_test_sm, y_pred)
utils.plot_confusion_matrix(y_test_sm, y_pred, attack_labels, "Ma trận nhầm lẫn của dữ liệu chưa giảm chiều - Mô hình Random forest" )

In [ ]:
accuracy = accuracy_score(y_test_sm,y_pred)
print("Accuracy:", accuracy)

report = classification_report(y_test_sm,y_pred, output_dict=True)
print("Precision (macro):", report["macro avg"]["precision"])
print("Recall (macro):", report["macro avg"]["recall"])
print("F1-score (macro):", report["macro avg"]["f1-score"])
print("Support (macro):", report["macro avg"]["support"])

MLP

In [ ]:
model_mlp = model.model_mlp(X_train_sm, y_train_sm, input_size=X_train_sm.shape[1],
                  hidden_size1=128, hidden_size2=64, hidden_size3=32, output_size=12, epochs=10000)
y_pred = model.predict_mlp(model_mlp, X_test_sm)
utils.report(y_test_sm,y_pred)
utils.plot_confusion_matrix(y_test_sm,y_pred, attack_labels, "Ma trận nhầm lẫn của dữ liệu chưa giảm chiều - Mô hình MLP" )

In [ ]:
accuracy = accuracy_score(y_test_sm,y_pred)
print("Accuracy:", accuracy)

report = classification_report(y_test_sm,y_pred, output_dict=True)
print("Precision (macro):", report["macro avg"]["precision"])
print("Recall (macro):", report["macro avg"]["recall"])
print("F1-score (macro):", report["macro avg"]["f1-score"])
print("Support (macro):", report["macro avg"]["support"])

Train với dữ liệu đã giảm chiều

In [ ]:
# Xử lý dữ liệu
list = DataProcessing.select_feature(X_train_sm, y_train_sm)
print(len(list))

In [ ]:
X_train_sm = X_train_sm[list['Feature']]
df_reduced, dropped_columns = DataProcessing.remove_highly_correlated_columns(X_train_sm[list['Feature']], threshold=0.8)

In [ ]:
X_train_sm_dr = df_reduced
X_test_sm_dr = X_test_sm[list['Feature']].drop(dropped_columns, axis=1)
y_train_sm_dr = y_train_sm.copy()
y_test_sm_dr = y_test_sm.copy()

LinearSVC

In [ ]:
model_LinearSVC_dr = model.model_LinearSVC(X_train_sm_dr, y_train_sm_dr)
y_pred_dr = model_LinearSVC_dr.predict(X_test_sm_dr)
utils.report(y_test_sm_dr,y_pred_dr)
utils.plot_confusion_matrix(y_test_sm_dr,y_pred_dr, attack_labels, "Ma trận nhầm lẫn của dữ liệu đã giảm chiều - Mô hình LinearSVC" )

In [ ]:
accuracy = accuracy_score(y_test_sm_dr, y_pred_dr)
print("Accuracy:", accuracy)

report = classification_report(y_test_sm_dr, y_pred_dr, output_dict=True)
print("Precision (macro):", report["macro avg"]["precision"])
print("Recall (macro):", report["macro avg"]["recall"])
print("F1-score (macro):", report["macro avg"]["f1-score"])
print("Support (macro):", report["macro avg"]["support"])

xgboost

In [ ]:
model_xgboost_dr = model.model_xgboost(X_train_sm_dr, y_train_sm_dr,12)
y_pred_dr = model_xgboost_dr.predict(X_test_sm_dr)
utils.report(y_test_sm_dr,y_pred_dr)
utils.plot_confusion_matrix(y_test_sm_dr,y_pred_dr, attack_labels, "Ma trận nhầm lẫn của dữ liệu đã giảm chiều - Mô hình XGBoost" )

In [ ]:
accuracy = accuracy_score(y_test_sm_dr,y_pred_dr)
print("Accuracy:", accuracy)

report = classification_report(y_test_sm_dr,y_pred_dr, output_dict=True)
print("Precision (macro):", report["macro avg"]["precision"])
print("Recall (macro):", report["macro avg"]["recall"])
print("F1-score (macro):", report["macro avg"]["f1-score"])
print("Support (macro):", report["macro avg"]["support"])

logistic_regression

In [ ]:
model_logistic_regression_dr = model.model_logistic_regression(X_train_sm_dr, y_train_sm_dr)
y_pred_dr = model_logistic_regression_dr.predict(X_test_sm_dr)
utils.report(y_test_sm_dr,y_pred_dr)
utils.plot_confusion_matrix(y_test_sm_dr,y_pred_dr, attack_labels, "Ma trận nhầm lẫn của dữ liệu chưa đã chiều - Mô hình Logistic regression" )

In [ ]:
accuracy = accuracy_score(y_test_sm_dr,y_pred_dr)
print("Accuracy:", accuracy)

report = classification_report(y_test_sm_dr,y_pred_dr, output_dict=True)
print("Precision (macro):", report["macro avg"]["precision"])
print("Recall (macro):", report["macro avg"]["recall"])
print("F1-score (macro):", report["macro avg"]["f1-score"])
print("Support (macro):", report["macro avg"]["support"])

knn

In [ ]:
model_knn_dr = model.model_knn(X_train_sm_dr, y_train_sm_dr)
y_pred_dr = model_knn_dr.predict(X_test_sm_dr)
utils.report(y_test_sm_dr, y_pred_dr)
utils.plot_confusion_matrix(y_test_sm_dr, y_pred_dr, attack_labels, "Ma trận nhầm lẫn của dữ liệu đã giảm chiều - Mô hình KNN" )

In [ ]:
accuracy = accuracy_score(y_test_sm_dr, y_pred_dr)
print("Accuracy:", accuracy)

report = classification_report(y_test_sm_dr, y_pred_dr, output_dict=True)
print("Precision (macro):", report["macro avg"]["precision"])
print("Recall (macro):", report["macro avg"]["recall"])
print("F1-score (macro):", report["macro avg"]["f1-score"])
print("Support (macro):", report["macro avg"]["support"])

random_forest

In [ ]:
model_random_forest_dr = model.model_random_forest(X_train_sm_dr, y_train_sm_dr)
y_pred_dr = model_random_forest_dr.predict(X_test_sm_dr)
utils.report(y_test_sm_dr, y_pred_dr)
utils.plot_confusion_matrix(y_test_sm_dr, y_pred_dr, attack_labels, "Ma trận nhầm lẫn của dữ liệu đã giảm chiều - Mô hình Random forest" )

In [ ]:
accuracy = accuracy_score(y_test_sm_dr, y_pred_dr)
print("Accuracy:", accuracy)

report = classification_report(y_test_sm_dr, y_pred_dr, output_dict=True)
print("Precision (macro):", report["macro avg"]["precision"])
print("Recall (macro):", report["macro avg"]["recall"])
print("F1-score (macro):", report["macro avg"]["f1-score"])
print("Support (macro):", report["macro avg"]["support"])

MLP

In [ ]:
model_mlp_dr = model.model_mlp(X_train_sm_dr, y_train_sm_dr, input_size=X_train_sm_dr.shape[1],
                  hidden_size1=128, hidden_size2=64, hidden_size3=32, output_size=12, epochs=10000)

In [ ]:
y_pred_dr = model.predict_mlp(model_mlp_dr, X_test_sm_dr)
utils.report(y_test_sm, y_pred_dr)
utils.plot_confusion_matrix(y_test_sm, y_pred_dr, attack_labels, "Ma trận nhầm lẫn của dữ liệu đã giảm chiều - Mô hình MLP" )

In [ ]:
accuracy = accuracy_score(y_test_sm, y_pred_dr)
print("Accuracy:", accuracy)

report = classification_report(y_test_sm, y_pred_dr, output_dict=True)
print("Precision (macro):", report["macro avg"]["precision"])
print("Recall (macro):", report["macro avg"]["recall"])
print("F1-score (macro):", report["macro avg"]["f1-score"])
print("Support (macro):", report["macro avg"]["support"])

Train mô hỉnh tinh chỉnh.

In [ ]:

# y_pred = model_random_forest.predict(X_test_sm)
# utils.report(y_pred,y_test_sm)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint

Random Forest

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
import time

# Không gian tham số mở rộng
param_dist = {
    'n_estimators': randint(50, 200),
    'max_depth': randint(3, 20),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 10),
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False]
}

# Khởi tạo mô hình Random Forest
get_random_forest_model_instance = model.get_random_forest_model() 

# Tìm kiếm tham số ngẫu nhiên
random_search = RandomizedSearchCV(
    estimator=get_random_forest_model_instance,
    param_distributions=param_dist,
    n_iter=50,
    cv=5,
    scoring='f1_macro',
    random_state=42,
    n_jobs=-1,
    verbose=1
)

# Huấn luyện và đo thời gian
start_time = time.time()
random_search.fit(X_train_sm_dr, y_train_sm_dr)
end_time = time.time()
print(f'Thời gian huấn luyện: {end_time - start_time:.2f} giây')

# Kết quả
print("Best params:", random_search.best_params_)

# Dự đoán và đánh giá
y_pred_rd = random_search.predict(X_test_sm_dr)
print("Random Forest Classification Report:")
utils.report(y_test_sm_dr, y_pred_rd)

utils.plot_confusion_matrix(
    y_test_sm_dr,
    y_pred_rd,
    attack_labels,
    "Ma trận nhầm lẫn trên dữ liệu đã giảm chiều - Mô hình Random Forest (RandomizedSearchCV)"
)


In [ ]:
accuracy = accuracy_score(y_test_sm_dr,y_pred_rd)
print("Accuracy:", accuracy)

report = classification_report(y_test_sm_dr,y_pred_rd, output_dict=True)
print("Precision (macro):", report["macro avg"]["precision"])
print("Recall (macro):", report["macro avg"]["recall"])
print("F1-score (macro):", report["macro avg"]["f1-score"])
print("Support (macro):", report["macro avg"]["support"])

XGBoost

In [ ]:


# Kiểm tra phân bố lớp
print("Class distribution:", Counter(y_train_sm_dr))

# Tính scale_pos_weight
class_counts = np.bincount(y_train_sm_dr)
scale_pos_weight = float(class_counts[0]) / class_counts[1]

# Không gian tham số mở rộng
param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'scale_pos_weight': [1.0, scale_pos_weight],
    'gamma': [0, 0.1, 0.2]  # tuỳ chọn
}

# Khởi tạo mô hình
xgb_model = model.get_xgboost_model(random_state=42)

# Tìm kiếm ngẫu nhiên
random_search_xgb = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_dist,
    n_iter=50,  # tăng nếu có thể
    cv=5,
    scoring='f1_macro',
    random_state=42,
    n_jobs=-1,
    verbose=1
)

# Huấn luyện
start_time = time.time()
random_search_xgb.fit(X_train_sm_dr, y_train_sm_dr)
end_time = time.time()
print(f'Thời gian huấn luyện: {end_time - start_time:.2f} giây')

# Mô hình tốt nhất
best_xgb_model = random_search_xgb.best_estimator_
print("Best parameters for XGBoost:", random_search_xgb.best_params_)

# Đánh giá
y_pred_xgb = best_xgb_model.predict(X_test_sm_dr)
print("XGBoost Classification Report:")
utils.report(y_test_sm_dr, y_pred_xgb)
utils.plot_confusion_matrix(
    y_test_sm_dr,
    y_pred_xgb,
    attack_labels,
    "Ma trận nhầm lẫn trên dữ liệu đã giảm chiều - Mô hình XGBoost (RandomizedSearchCV)"
)


In [ ]:
accuracy = accuracy_score(y_test_sm_dr, y_pred_xgb)
print("Accuracy:", accuracy)

report = classification_report(y_test_sm_dr, y_pred_xgb, output_dict=True)
print("Precision (macro):", report["macro avg"]["precision"])
print("Recall (macro):", report["macro avg"]["recall"])
print("F1-score (macro):", report["macro avg"]["f1-score"])
print("Support (macro):", report["macro avg"]["support"])

KNN

In [ ]:
# Định nghĩa không gian tham số cho KNN
param_dist_knn = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

# Khởi tạo mô hình
knn_model = model.get_knn_model()

# Tìm kiếm ngẫu nhiên
random_search_knn = RandomizedSearchCV(
    estimator=knn_model,
    param_distributions=param_dist_knn,
    n_iter=50,
    cv=5,
    scoring='f1_macro',
    random_state=42,
    n_jobs=-1
)

# Huấn luyện
start_time = time.time()
random_search_knn.fit(X_train_sm_dr, y_train_sm_dr)
end_time = time.time()
print(f'Thời gian huấn luyện: {end_time - start_time:.2f} giây')
# Lấy mô hình tốt nhất
best_knn_model = random_search_knn.best_estimator_
print("Best parameters for KNN:", random_search_knn.best_params_)

# Dự đoán và đánh giá
y_pred_knn = best_knn_model.predict(X_test_sm_dr)
print("KNN Classification Report:")
utils.report(y_test_sm_dr, y_pred_knn)
utils.plot_confusion_matrix(y_test_sm_dr, y_pred_knn, attack_labels, "Ma trận nhầm lẫn trên dữ liệu đã giảm chiều - Mô hình KNN (RandomizedSearchCV)" )

In [ ]:
accuracy = accuracy_score(y_test_sm_dr, y_pred_knn)
print("Accuracy:", accuracy)

report = classification_report(y_test_sm_dr, y_pred_knn, output_dict=True)
print("Precision (macro):", report["macro avg"]["precision"])
print("Recall (macro):", report["macro avg"]["recall"])
print("F1-score (macro):", report["macro avg"]["f1-score"])
print("Support (macro):", report["macro avg"]["support"])
